In [88]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')   ## set French as local language, useful for date display
# locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # non-windows computer? 

import pandas as pd

In [89]:
# Load json data
# --------------
import json
import emoji

# deal with telegram messages
import re


# in dataframe
#df = pd.read_json('messages/inbox/marcnegre_hwizlpvhxw/message_1.json')
#df.info()


# in dictionnary
#with open('telegram/ChatExport_2020-11-15/result.json') as f:
#    test = json.load(f)

In [100]:
raw_data_telegram = pd.read_json('telegram/ChatExport_2020-11-15/result.json')
raw_data_telegram.info()
raw_data_telegram.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11826 entries, 0 to 11825
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      11826 non-null  object
 1   type      11826 non-null  object
 2   id        11826 non-null  int64 
 3   messages  11826 non-null  object
dtypes: int64(1), object(3)
memory usage: 369.7+ KB


,name,type,id,messages
0,Marc,personal_chat,4957211864,"{'id': 591901, 'type': 'service', 'date': '201..."
1,Marc,personal_chat,4957211864,"{'id': 676784, 'type': 'message', 'date': '202..."
2,Marc,personal_chat,4957211864,"{'id': 676785, 'type': 'message', 'date': '202..."
3,Marc,personal_chat,4957211864,"{'id': 676917, 'type': 'message', 'date': '202..."
4,Marc,personal_chat,4957211864,"{'id': 679644, 'type': 'message', 'date': '202..."


In [101]:
raw_data_telegram["messages"][6]
print(type(raw_data_telegram["messages"][6]['text']))
raw_data_telegram["messages"][6]['text'][0]['text']


<class 'list'>


'#sage'

In [102]:
def preprocess_telegram_msg(text: str):
    """Process text message for latex, adapt bold, italic, code and underline delimiters"""
    # telegram formating documentation: https://core.telegram.org/api/entities
    regexs = [
        (r"\*\*\*(:?.*?(?<!\*))\*\*\*", r"\\textbf{\\textit{\g<1>}}")  # bold & italic
        ,(r"___(:?.*?(?<!_))___", r"\\textbf{\\textit{\g<1>}}")  # bold & italic
        ,(r"\*\*(:?.*?(?<!\*))\*\*", r"\\textbf{\g<1>}")  # bold
        ,(r"__(:?.*?(?<!_))__", r"\\textbf{\g<1>}")  # bold
        ,(r"<b>(:?.*?)<\\b>", r"\\textbf{\g<1>}")  # bold
        ,(r"<strong>(:?.*?)<\\strong>", r"\\textbf{\g<1>}")  # bold
        ,(r"\*(:?.*?(?<!\*))\*", r"\\textit{\g<1>}")  # italic
        ,(r"_(:?.*?(?<!_))_", r"\\textit{\g<1>}")  # italic
        ,(r"<i>(:?.*?)<\\i>", r"\\textit{\g<1>}")  # italic
        ,(r"<em>(:?.*?)<\\em>", r"\\textit{\g<1>}")  # italic
        ,(r"<u>(:?.*?)<\\u>", r"\\underline{\g<1>}")  # underline
        ,(r"<code>(:?.*?)<\\code>", r"\\texttt{\g<1>}")  # code
    ]  # Beware, regex sustitution order is important

    for regex, subst in regexs:
      text = re.sub(regex, subst, text, 0, re.MULTILINE)
    
    return text

In [106]:
raw_data_telegram["messages"][1]

{'id': 676784,
 'type': 'message',
 'date': '2020-06-16T18:01:09',
 'from': 'Bénédicte Colnet',
 'from_id': 4555368440,
 'text': "La pandémie du covid-19 est une maladie nouvelle pour laquelle la recherche de traitement a remis au coeur du débat - médiatique notamment - le processus d’évaluation d’un médicament. Alors que l’un des seuls moyens de sauver des patients semble être de placer les personnes gravement atteintes en salle de réanimation et sous respirateurs, la recherche de traitement a suscité un débat mondial avec plusieurs molécules au centre du jeu, en particulier l’hydroxychloroquine. Réutiliser une molécule déjà sur le marché, mais pour traiter une autre maladie s’appelle le repurposing, ce qui est donc le cas des nombreuses molécules proposées pour traiter la covid-19. Lors de cette pandémie l’opinion publique et la sphère politique se sont approprié la question de “Faut-il ou non utiliser l'hydroxychloroquine ?”. Pourtant cette question est usuellement celle de l’expert

In [146]:
concatenated_table = {'source': [], # telegram or messenger (string)
        'datetime': [], # date (string)
        'sender': [], # M or B (string)
        'message': [], # content in text (string)
        'path': []} # path to images or gifs (list of string)

concatenated_table = pd.DataFrame(concatenated_table)

# Deal with telegram messages
for element in raw_data_telegram["messages"][1:30]:
    
    # simplify sender
    if element["from"] == 'Marc Negre':
        sender = "M"
    elif element["from"] == 'Bénédicte Colnet':
        sender = "B"
    else:
        print("Error on sender on message number "+str(element))
        break
    
    # Deal with picture (apparently several pictures are send as several messages)
    if 'photo' in element:
        photo = element["photo"]
    else:
        photo = []
        
    
    # deal with text that are list or hastag
    if type(element["text"]) == str:
        text = element["text"]
    elif type(element["text"]) == list:
        text = element["text"][0]
        if type(text) == dict:
            text = "\\" + element['text'][0]['text']
    else:
        print("Error in the type of text")
        break
        
    # deal with emoji
    new_text = ""
    for c in text:
        if not(c in emoji.UNICODE_EMOJI):
            new_text = new_text + c
        else:
            new_text = new_text + "\emoji[ios]{"+f"{hex(ord(c))[2:]}"+"}" #Change for f string instead of [2:]
            
    # deal with formatting
    new_text = preprocess_telegram_msg(new_text)
    
    # replace \n with \\
    new_text = new_text.replace("\n", "\\ ")
    
    new_row = {'source' : "Telegram", 'datetime': element["date"], 'sender' : sender, 'message': new_text, 'path': photo}
    concatenated_table = concatenated_table.append(new_row, ignore_index = True)
    

In [147]:
time_col_name = 'datetime'

# format datetime
concatenated_table[time_col_name] = pd.to_datetime(concatenated_table[time_col_name])

# sort by datetime
concatenated_table = concatenated_table.sort_values(time_col_name).reset_index(drop=True)

# add 'date' column
concatenated_table['date'] = concatenated_table[time_col_name].dt.date

# add columns for pretty date and time as strings
concatenated_table['dateStr'] = concatenated_table[time_col_name].apply(lambda x: x.strftime("%A %d %B").replace(' 0', ' ')) # "%A %d %B %Y": +year
concatenated_table['timeStr'] = concatenated_table[time_col_name].apply(lambda x: x.strftime("%H:%M"))


# add boolean colum specifying who is the "right" (as opposed to "left") sender
right_sender = concatenated_table['sender'].unique()[0]  # modify here
concatenated_table['right'] = (concatenated_table['sender'] == right_sender)

# add sender group id column (on a given date, helper column to handle consecutive messages by the same sender)
def f(concatenated_table):
    s = (concatenated_table['right'].shift() != concatenated_table['right'])
    s.iloc[0] = False
    return s.cumsum()

concatenated_table['senderGroup'] = concatenated_table.groupby('date', as_index=False).apply(f).reset_index(drop=True)


concatenated_table.head()

,source,datetime,sender,message,path,date,dateStr,timeStr,right,senderGroup
0,Telegram,2020-06-16 18:01:09,B,La pandémie du covid-19 est une maladie nouvel...,[],2020-06-16,Mardi 16 juin,18:01,True,0
1,Telegram,2020-06-16 18:06:17,M,Lois mathématiques --,[],2020-06-16,Mardi 16 juin,18:06,False,1
2,Telegram,2020-06-17 09:19:55,M,"J'ai lu le teasing d'intro, super bien posé, h...",[],2020-06-17,Mercredi 17 juin,09:19,False,0
3,Telegram,2020-07-07 15:54:05,B,Du coup si on résume : \ * \emoji[ios]{1f682} ...,[],2020-07-07,Mardi 7 juillet,15:54,True,0
4,Telegram,2020-07-18 00:16:11,B,j'ai meme fait le graphique de la médecin,photos/photo_1@18-07-2020_00-16-11.jpg,2020-07-18,Samedi 18 juillet,00:16,True,0


In [167]:
def left_formating(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.timeStr}&{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\noindent', '\\begin{tabular}{p{0.05\\textwidth}p{0.55\\textwidth}}'] +
        table_rows
        + ['\\end{tabular}'])
    
    return text

def left_formating_with_bubbles(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{leftbubbles}'] +
        table_rows
        + ['\\end{leftbubbles}']) 
    return text


def right_formating(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}&{x.timeStr}\\\\' if len(x.message) > 45 
                  else '\multicolumn{1}{r}' + f'{x.message}&{x.timeStr}\\\\',  # format for right rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{flushright}', '\\noindent', '\\begin{tabular}{p{0.55\\textwidth}p{0.1\\textwidth}}'] +
        table_rows
        + ['\\end{tabular}', '\\end{flushright}'])

    return text

def right_formating_with_bubbles(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{rightbubbles}'] +
        table_rows
        + ['\\end{rightbubbles}'])
    return text

# By default it will apply bubbles style
def format_msg(df, bubbles = True):
    if bubbles:
        if df['right'].unique()[0]:
            return right_formating_with_bubbles(df)
        return left_formating_with_bubbles(df)
    else:
        if df['right'].unique()[0]:
            return right_formating(df)
        return left_formating(df)

In [168]:
#print(left_formating(df))
#print(right_formating(df))

In [169]:
# Loop over dates, generate yyyy-mm-dd.tex section files, one per day
# -------------------------------------------------------------------

df = concatenated_table[2:20]


separate_messages = True

for date, df_t in df.groupby('date'):
    date_str = df_t['dateStr'].unique()[0]
    title = '\\section*{' + date_str + '}'
    
    if separate_messages:
        discussion = ' '.join(df_t.groupby('senderGroup').apply(format_msg))
    else:
        discussion = '\n\n'.join(df_t.groupby('senderGroup').apply(format_msg))
    
    text = f'{title}\n\n{discussion}'
    
    with open(f'./sections/{date}.tex', 'w', encoding='utf-8') as ft:
        ft.write(text)